In [65]:
import numpy as np
from numpy import linalg as la

# Generate a mixed state of randomly sampled pure qudit states                                                 
# d - onsite hilbert space dimension
# N - number of sites
# M - number of pure states used
def random_mixed_state(N,d=3,M=10):
        states = [random_state(N,d) for i in range(M)]
        weights = [np.random.uniform(low=0.0,high=1.0) for i in range(M)]
        total_weight = sum(weights)
        state = np.zeros((d**N,d**N))
        for i in range(M):
                state = state + (weights[i]/total_weight) * np.outer(states[i],states[i].conj())
        return state

# Generate a random qudit state
# d - onsite hilbert space dimension
# N - number of sites
def random_state(N,d=3):
        normals = [np.random.standard_normal(size=d**N) for i in range(2)]
        state = np.array([normals[0][i] + 1j * normals[1][i] for i in range(d**N)])
        state /= la.norm(state)
        return state

# Given a density matrix, return its matrix product operator representation
# Indices are stored in the following format: (bond,phys,bond)
def state_to_mpo(state, n, d, verbose=False, **kwargs):
    # If not given a maximum bond dimension, set it to the maximum possible - d^(4n) (TODO)?
    #TODO: add a truncation magnitude cuttoff instead of just max_bd
    max_bd = kwargs.get('max_bd', d**(4*n))
    
    # Constructing environment tensor                                                                              
    tensor_shape = tuple([d]*(2*n))
    tensor = state.reshape(tensor_shape)
    tensor_axes = [i for i in range(2*n)]
    T_axes = [int(i/2) if i%2 == 0 else int(n + (i/2)) for i in range(2*n)]
    d2 = d**2
    T = np.moveaxis(tensor, tensor_axes, T_axes)
    T = T.reshape((d2,d2**(n-1)))
    if verbose:
        print(0, "Initial T:\t\t\t\t", T.shape)

    # First site
    mpo = [0]*n
    U, S, Vt = la.svd(T, full_matrices = False)
    U = U[:,:max_bd]
    S = S[:max_bd]
    Vt = Vt[:max_bd,:]
    mpo[0] = U.reshape((1,U.shape[0],U.shape[1]))
    T = np.dot(np.diag(S),Vt).reshape(S.size*d2,int(Vt.shape[1]/d2))
    if verbose:
        print(1, U.shape, Vt.shape, "\t->", mpo[0].shape, "\t", T.shape, "\t", S.size)

    # Interior sites
    for i in range(1,n-1):
            U, S, Vt = la.svd(T, full_matrices = False)
            U = U[:,:max_bd]
            S = S[:max_bd]
            Vt = Vt[:max_bd,:]
            mpo[i] = U.reshape((int(U.shape[0]/d2),d2,U.shape[1]))
            T = np.dot(np.diag(S),Vt).reshape((S.size*d2,int(Vt.shape[1]/d2)))
            if verbose:
                print(i+1, U.shape, Vt.shape, "\t->", mpo[i].shape, "\t", T.shape, "\t", S.size)

    # Last site
    mpo[n-1] = np.dot(np.diag(S),Vt).reshape((S.size, Vt.shape[0], 1))
    if verbose:
        print(n, "Final Matrix:\t\t  ", mpo[n-1].shape)

    return mpo

# Frobenius inner product tr[A^\dagger B] between two states A and B represented by mpos 
# norm: Frobenius normalization of local basis elements used in MPO representation (norm = Tr[A_u^\dagger A_v])
# Assumptions:
#       len(m1) = len(m2)                                       [same length]
#       m1[i].shape = m2[i].shape = (bond,phys,bond)            [same bond and phys dim at each site]
#       m1[0].shape = (1,_,_) and m1[-1].shape = (_,_,1)        [closed boundary conditions]
def inner_prod(m1, m2, norm=1):
        # Extract length, phys dim, and first site bond dim
        n = len(m1)

        # First site
        M = np.tensordot(m1[0][0].conj(),m2[0][0],axes=([0,0]))      # sum over physical indices

        #print(0,m1[0].shape,M.shape)
        # Rest of contraction
        for i in range(1,n):
                M = np.tensordot(M,m1[i].conj(),axes=([0,0]))
                M = np.tensordot(M,m2[i],axes=([0,1],[0,1]))
                #print(i,m1[i].shape, M.shape)
        return M[0][0]*norm

In [66]:
d = 3
n = 5
rho = random_mixed_state(n,d)
m_rho = state_to_mpo(rho,n,d)
print(inner_prod(m_rho,m_rho), np.trace(np.dot(rho.conj().T,rho)))

(0.1404680921101127-1.3010426069826053e-18j) (0.1404680921101129+0j)


## Moving to phase space

We formed our MPO like so: $\rho_{ij}\left|i\right>\left<j\right|=\rho_{(i_1\cdots i_n)(j_1\cdots j_n)}(\left|i_1\right>\otimes \cdots \otimes \left|i_n\right>)(\left<j_1\right|\otimes \cdots \otimes \left<j_n\right|)=\sum_{i_kj_k\alpha _k}\rho^{(1)}_{\alpha_0(i_1j_1)\alpha_1}\left|i_1\right>\left<j_1\right|\otimes\cdots\otimes \rho^{(n)}_{\alpha_{n-1}(i_nj_n)\alpha_n}\left|i_n\right>\left<j_n\right|$. This gave us a matrix product operator of the form $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}}C_{u_k}$, where $C_u=\left|u_1\right>\left<u_2\right|$. We want to represent it in phase space, i.e. we want to find the coefficents $\tidle \rho$ satisfying $\sum_{u,\alpha} \otimes_k \rho^{(k)u_k}_{\alpha_k\alpha_{k+1}} C_{u_k}=\sum_{u,\alpha} \otimes_k \tilde{\rho}^{(k)u_k}_{\alpha_k\alpha_{k+1}} A_{u_k}$. The following choice works:

$$\tilde{\rho}^{(k)u}_{\alpha\beta}=d^{-1}\sum_v\rho^{(k)v}_{\alpha\beta}\mathrm{Tr}\left[C_uA_v\right].$$

Note that $\mathrm{Tr}[C_u A_v]=\sum_k \left<k|u_1\right>\left<u_2|A_v |k\right>=\left<u_1|A_v|u_2\right>=(A_v)_{u_1,u_2}$, so the change-of-basis coefficients are proportional to the matrix elements of the phase space operators. Since $\mathrm{Tr}[A_uA_v]=d\delta_{u,v}$, the phase space basis is not normalized. This means that we pick up a Hilbert space factor when computing inner products:

$$\mathrm{Tr}[\rho^\dagger \sigma]=d^n\left<\rho|\sigma\right>_A,$$

where $\left<\rho|\sigma\right>_A$ denotes the contraction of the matrix product representations of the states $\rho$ and $\sigma$ in the phase space basis $A_u$.

In [100]:
import cmath
import math

d = 3   # qutrit

# qutrit X and Z matrices
omega = cmath.exp(cmath.pi*2j/d)
X = np.array([[0,0,1],[1,0,0],[0,1,0]])
Z = np.array([[1,0,0],[0,omega,0],[0,0,omega**2]])

# 1-qudit pauli matrices
T = np.array([[omega**(-2*a*b)*np.dot(la.matrix_power(Z,a),la.matrix_power(X,b)) for b in range(d)] for a in range(d)])

# 1-qudit phase space operators (TODO: change the sum below to a numpy sum)
A_0 = (1./d)*sum([sum([T[a,b] for b in range(d)]) for a in range(d)])
A = np.zeros((d,d,d,d), dtype=np.complex128)
for i in range(d):
    for j in range(d):
        A[i,j] = np.dot(T[i,j],np.dot(A_0,np.conj(T[i,j].T)))
A = A.reshape((d**2,d,d))

# Computational basis C_u = |u_1><u_2|
#comp_basis = np.zeros((d,d,d,d), dtype=np.complex128)
#for i in range(d):
#    for j in range(d):
#        comp_basis[i,j,i,j] = 1
#comp_basis = comp_basis.reshape((d**2,d,d))

# Change of basis coefficients Tr[A_u^\dagger C_u] = Tr[A_u C_u]
#ps_coefficients = np.zeros((d**2,d**2),dtype=np.complex128)
#for i in range(d**2):
#    for j in range(d**2):
#        ps_coefficients[i,j] = 1./d*np.trace(np.dot(A[i],comp_basis[j]))
ps_coefficients = 1./d*A.reshape((d**2,d**2))

# change of basis coefficients
# TODO: change this to a numpy array
#ps_coefficients = [[np.trace(np.dot(A[u],comp_basis[v])) for u in range(d**2)] for v in range(d**2)]

#TODO: get rid of for loops
def basis_change(mpo, coefficients):
    n = len(mpo)
    _,d,_ = mpo[0].shape
    
    res = [None]*n
    for k in range(n):
        res[k] = np.zeros(mpo[k].shape, dtype=np.complex128)
        temp = np.tensordot(mpo[k],coefficients,axes=([1,1])) # sum over physical indices
        temp = np.moveaxis(temp, [0,1,2], [0,2,1])
        res[k] = temp
    return res

In [101]:
# Check properties of basis elements
for i in range(d**2):
    for j in range(d**2):
        if abs(np.trace(np.dot(A[i],A[j])).imag) > 1e-14:
            print(i,j,"complex trace error!")
        if not(np.allclose(A[i].conj().T,A[i])):
            print(i,j,"hermiticity error!")
        if i == j:
            if abs(np.trace(np.dot(A[i],A[j])).real - d) > 1e-14:
                print(i,j,"normalization error!")
        else:
            if abs(np.trace(np.dot(A[i],A[j])).real) > 1e-14:
                print(i,j,"normalization error!")

# Check that inner products still work
print(inner_prod(m_rho,m_rho))
m_rho_ps = basis_change(m_rho, ps_coefficients)
print(inner_prod(m_rho_ps,m_rho_ps,norm=d**n))

(0.1404680921101127-1.3010426069826053e-18j)
(0.14046809211011266-1.6466320494623599e-18j)


In [5]:
import scipy as sp
from scipy.optimize import minimize

#TODO: seed 

def dist_sq(A, B, norm):
    return inner_prod(A,A) - 2*inner_prod(A,B) + inner_prod(B,B)

def cost_fct(i, x, A, B, norm):
        A[i] = x.reshape(A[i].shape)
        return inner_prod(A, B, norm)
    
def optimize(A, B, nswp = 20, opt_method = 'L-BFGS-B'):
        # The true distance between B and the positive-repr states is magik2:
        true_dist = magik(mpo_to_cfnts(B),2,2)
        print("Initial distance: %g, True distance: %g" % (dist(A,B), true_dist))

        # Optimize D(A||B) on A to find closest positive repr MPO to B
        print("Optimization method: %s" % opt_method)
        print("Sigma bond dimension: %d" % D_A)
        print("Half-sweep D(A||B) D(A||B)-D(A*||B) E(A) H_2(A)\n")
        sys.stdout.flush()

        n = len(A)
        _,d,_ = A.shape
        dn = d**n
        b = [(0, None) for i1 in range(D_A*(d**2))]
        start_time = time.time()
        for i1 in range(nswp):
                x0 = minimize(lambda x:cost_fct(0,x,A,B,dn), x0, method=opt_method, bounds=b).x
                A[0] = x0.reshape(A[0].shape)
                print("%d %g %g %g %g" % (2*i1+1, dist(A,B,dn), dist(A,B,dn)-true_dist, vn_entropy(A), r_entropy(A,2)))
                sys.stdout.flush()
                x1 = minimize(lambda x:cost_fct(1,x,A,B,dn), x1, method=opt_method, bounds=b).x
                A[1] = x1.reshape(A[1].shape)
                
                print("%d %g %g %g %g" % (2*i1+2, dist(A,B,dn), dist(A,B,dn)-true_dist, vn_entropy(A), r_entropy(A,2)))
                sys.stdout.flush()
                
        print("\nTime elapsed during optimization: %g" % (time.time() - start_time))